**1. How do you install Detectron2 using pip and check the version of Detectron2.**

In [1]:
pip install 'git+https://github.com/facebookresearch/detectron2.git'


  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-wcbj044s
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-wcbj044s
  Resolved https://github.com/facebookresearch/detectron2.git to commit c69939aa85460e8135f40bce908a6cddaa73065f
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.2/79.2 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 63.8 MB/s eta 0:00:00
  Created wheel for detectron2: filename=detectron2-0.6-cp3

**2. How do you perform inference with Detectron2 using an online image**

In [2]:
pip install requests Pillow

In [ ]:

import requests
from PIL import Image
import numpy as np
import torch
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

# Step 1: Download an online image
image_url = "/content/person.jpeg"  # Replace with the actual image URL
response = requests.get(image_url)
image = Image.open(BytesIO(response.content))

# Convert image to numpy array
image_np = np.asarray(image)

# Step 2: Configure the Detectron2 model
cfg = get_cfg()
cfg.merge_from_file("detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Adjust path as needed
cfg.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Set threshold for prediction score
cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

predictor = DefaultPredictor(cfg)

# Step 3: Perform inference
outputs = predictor(image_np)

# Step 4: Visualize results
v = Visualizer(image_np[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

# Convert visualization to a PIL image and display it
output_image = Image.fromarray(out.get_image()[:, :, ::-1])
output_image.show()


**3. How do you visualize evaluation metrics in Detectron2, such as training loss**

In [6]:
pip install tensorboard


In [ ]:
from detectron2.utils.logger import setup_logger
from detectron2.engine import DefaultTrainer

setup_logger()
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt

# Path to metrics.json
metrics_file = "./output/metrics.json"

# Load metrics
with open(metrics_file, "r") as f:
    metrics = [json.loads(line) for line in f]

# Convert to DataFrame
metrics_df = pd.DataFrame(metrics)

# Plot training loss
plt.figure(figsize=(10, 6))
metrics_df.plot(x="iteration", y="total_loss", kind="line", title="Training Loss")
plt.xlabel("Iterations")
plt.ylabel("Total Loss")
plt.grid()
plt.show()

# Plot other metrics if available
if "bbox/AP" in metrics_df.columns:
    plt.figure(figsize=(10, 6))
    metrics_df.plot(x="iteration", y="bbox/AP", kind="line", title="Bounding Box Average Precision (AP)")
    plt.xlabel("Iterations")
    plt.ylabel("AP")
    plt.grid()
    plt.show()


**4. How do you run inference with TFOD2 on an online image**

In [ ]:
import tensorflow as tf
import numpy as np
from PIL import Image
import requests
from io import BytesIO

# Step 1: Load the pre-trained model
model_dir = "path/to/saved_model"  # Replace with the path to your saved model
detection_model = tf.saved_model.load(model_dir)

# Step 2: Define a function for inference
def run_inference(model, image_np):
    # Convert image to a tensor and add a batch dimension
    input_tensor = tf.convert_to_tensor(image_np)
    input_tensor = input_tensor[tf.newaxis, ...]

    # Run the model
    detections = model(input_tensor)

    # Convert detections to numpy arrays
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy() for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # Convert detection classes to integers
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    return detections

# Step 3: Download and preprocess the online image
image_url = "https://example.com/path/to/your/image.jpg"  # Replace with an actual image URL
response = requests.get(image_url)
image = Image.open(BytesIO(response.content))

# Convert image to a numpy array
image_np = np.array(image)

# Step 4: Perform inference
detections = run_inference(detection_model, image_np)

# Step 5: Display the detections
# Draw bounding boxes and labels on the image
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

def visualize_detections(image_np, detections, threshold=0.5):
    fig, ax = plt.subplots(1, figsize=(12, 8))
    ax.imshow(image_np)

    for i in range(len(detections['detection_scores'])):
        score = detections['detection_scores'][i]
        if score >= threshold:
            # Get bounding box
            bbox = detections['detection_boxes'][i]
            ymin, xmin, ymax, xmax = bbox

            # Scale box to image size
            im_height, im_width, _ = image_np.shape
            (xmin, xmax, ymin, ymax) = (xmin * im_width, xmax * im_width, ymin * im_height, ymax * im_height)

            # Draw rectangle
            rect = Rectangle((xmin, ymin), xmax - xmin, ymax - ymin, linewidth=2, edgecolor='r', facecolor='none')
            ax.add_patch(rect)

            # Draw label
            class_id = detections['detection_classes'][i]
            ax.text(xmin, ymin - 10, f"Class {class_id}: {score:.2f}", color='red', fontsize=12, backgroundcolor='white')

    plt.show()

visualize_detections(image_np, detections)


**5. How do you install TensorFlow Object Detection API in Jupyter Notebook**

In [12]:
pip install tensorflow


In [ ]:
git clone https://github.com/tensorflow/models.git

In [ ]:
import os
os.environ['PYTHONPATH'] += f":{os.getcwd()}:{os.getcwd()}/slim"


In [ ]:
!pip install .


In [ ]:
!python object_detection/builders/model_builder_tf2_test.py

In [ ]:
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

**6. How can you load a pre-trained TensorFlow Object Detection model**

In [ ]:
import tensorflow as tf
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# Load the model
model_dir = "path/to/saved_model"
detection_model = tf.saved_model.load(model_dir)

# Load an image
image_path = "path/to/your/image.jpg"
image = Image.open(image_path)
image_np = np.array(image)

# Perform inference
input_tensor = tf.convert_to_tensor(image_np)
input_tensor = input_tensor[tf.newaxis, ...]
detections = detection_model(input_tensor)

# Visualize detections
def visualize_detections(image_np, detections, threshold=0.5):
    import matplotlib.patches as patches

    plt.figure(figsize=(12, 8))
    plt.imshow(image_np)
    ax = plt.gca()

    for i in range(int(detections['num_detections'][0])):
        score = detections['detection_scores'][0][i]
        if score > threshold:
            bbox = detections['detection_boxes'][0][i].numpy()
            ymin, xmin, ymax, xmax = bbox
            im_height, im_width, _ = image_np.shape
            (xmin, xmax, ymin, ymax) = (xmin * im_width, xmax * im_width, ymin * im_height, ymax * im_height)

            # Draw bounding box
            rect = patches.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin, linewidth=2, edgecolor='r', facecolor='none')
            ax.add_patch(rect)

            # Label the box
            ax.text(xmin, ymin - 10, f"Score: {score:.2f}", color='red', fontsize=12, backgroundcolor='white')

    plt.show()

visualize_detections(image_np, detections)


** 7.How do you preprocess an image from the web for TFOD2 inference**

In [ ]:
import tensorflow as tf
import numpy as np
from PIL import Image
import requests
from io import BytesIO

# Step 1: Fetch the image from the web
image_url = "https://example.com/path/to/image.jpg"  # Replace with an actual image URL
response = requests.get(image_url)
image = Image.open(BytesIO(response.content))

# Step 2: Resize the image (if required)
input_size = (320, 320)  # Replace with the input size of your model
image_resized = image.resize(input_size)

# Step 3: Convert to NumPy array
image_np = np.array(image_resized)

# Step 4: Add a batch dimension
input_tensor = tf.convert_to_tensor(image_np)
input_tensor = input_tensor[tf.newaxis, ...]

# The input_tensor is now ready for inference
print(f"Preprocessed image shape: {input_tensor.shape}")


**8. How do you visualize bounding boxes for detected objects in TFOD2 inference.**

In [ ]:
import numpy as np
import tensorflow as tf
from PIL import Image
import matplotlib.pyplot as plt
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import label_map_util


In [ ]:
# Assume `detection_model` is your loaded TFOD2 model
# `image_np` is the input image as a NumPy array

# Add batch dimension
input_tensor = tf.convert_to_tensor(image_np)[tf.newaxis, ...]

# Run inference
detections = detection_model(input_tensor)

# Convert tensors to NumPy arrays
detections = {key: value.numpy() for key, value in detections.items()}
detections['num_detections'] = int(detections['num_detections'][0])
detections['detection_classes'] = detections['detection_classes'][0].astype(np.int64)
detections['detection_boxes'] = detections['detection_boxes'][0]
detections['detection_scores'] = detections['detection_scores'][0]


In [ ]:
# Path to the label map file
label_map_path = "path/to/label_map.pbtxt"

# Load the label map
category_index = label_map_util.create_category_index_from_labelmap(label_map_path, use_display_name=True)


In [ ]:
# Visualize the detections on the image
image_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
    image_with_detections,
    detections['detection_boxes'],
    detections['detection_classes'],
    detections['detection_scores'],
    category_index,
    use_normalized_coordinates=True,
    max_boxes_to_draw=20,         # Maximum number of boxes to draw
    min_score_thresh=0.5,         # Minimum confidence threshold
    agnostic_mode=False
)

# Display the image with detections
plt.figure(figsize=(12, 8))
plt.imshow(image_with_detections)
plt.axis('off')
plt.show()


**10. How do you define classes for custom training in TFOD2.**

In [ ]:
fine_tune_checkpoint: "path/to/pretrained_model_checkpoint/model.ckpt"
num_classes: 2  # Change this to your number of classes
train_input_path: "path/to/train.record"
eval_input_path: "path/to/eval.record"
label_map_path: "path/to/label_map.pbtxt"


In [ ]:
python3 models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path=path/to/your/pipeline.config \
    --model_dir=path/to/save/model \
    --checkpoint_dir=path/to/save/model \
    --eval_timeout=0


In [ ]:
python3 models/research/object_detection/exporter_main_v2.py \
    --input_type image_tensor \
    --pipeline_config_path=path/to/your/pipeline.config \
    --trained_checkpoint_dir=path/to/save/model \
    --output_directory=path/to/exported_model


**11. How do you resize an image before detecting object1**

In [ ]:
from object_detection.utils import visualization_utils as vis_util
from object_detection.utils import label_map_util

# Load the label map (class names)
label_map_path = 'path/to/label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(label_map_path, use_display_name=True)

# Visualize the detections on the image
image_with_detections = image_np.copy()

vis_util.visualize_boxes_and_labels_on_image_array(
    image_with_detections,
    output_dict['detection_boxes'][0].numpy(),
    output_dict['detection_classes'][0].numpy().astype(np.int64),
    output_dict['detection_scores'][0].numpy(),
    category_index,
    use_normalized_coordinates=True,
    max_boxes_to_draw=20,
    min_score_thresh=0.5,
    agnostic_mode=False
)

# Display the image with bounding boxes
plt.figure(figsize=(10, 10))
plt.imshow(image_with_detections)
plt.axis('off')
plt.show()


**12.  How can you apply a color filter (e.g., red filter) to an image?**

In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
# Convert the image to a NumPy array
image_np = np.array(image)

# Check the shape of the image
print("Image shape:", image_np.shape)  # (height, width, channels)


In [ ]:
# Apply the red filter by setting the green and blue channels to zero
image_with_red_filter = image_np.copy()
image_with_red_filter[:, :, 1] = 0  # Set green channel to 0
image_with_red_filter[:, :, 2] = 0  # Set blue channel to 0


In [ ]:
# Convert the NumPy array back to a PIL image
filtered_image = Image.fromarray(image_with_red_filter)

# Show the filtered image
filtered_image.show()


In [ ]:
# Save the filtered image
filtered_image.save('path_to_save_filtered_image.jpg')  # Replace with your desired path
